# Test `pipelineGetData.js`

In [6]:
!node ../src/pipelineGetData.js

SyntaxError: Unexpected token '.'

In [5]:
!echo "hello"

SyntaxError: Unexpected string

In [1]:
console.log(`Running pipelineGetData.js`);
const {loadJsonFile, saveResponseJson} = require('../src/fileFunctions');
const nf = require('../src/notionFunctions');
// const period = 'month';
// const jsonFileName = `../data/notionTimeTracking_${period}`;
// const parsedJsonFileName = `../data/notion_time_tracking_parsed_${period}`;

// const response = await nf.queryNotionAndSaveResponse(period, jsonFileName, save=true);
// const data = loadJsonFile(`${jsonFileName}.json`);
// const parsedData = parseTimeTracking(data, save=true, parsedJsonFileName);
// console.log(`Done!`);


async function main() {
  const period = 'month';
  const jsonFileName = `../data/notionTimeTracking_${period}`;
  const parsedJsonFileName = `../data/notion_time_tracking_parsed_${period}`;

  const response = await nf.queryNotionAndSaveResponse(period, jsonFileName, save=true);
  const data = loadJsonFile(`${jsonFileName}.json`);
  const parsedData = parseTimeTracking(data, save=true, parsedJsonFileName);
  console.log(`Done!`);

}

main().catch(error => {
  console.error('Error:', error);
});

Running pipelineGetData.js
Current time stamp: 2023-12-26_1439


Promise { <pending> }

Error: TypeError [ERR_INVALID_CALLBACK]: Callback must be a function. Received undefined
    at maybeCallback (node:fs:178:3)
    at Object.writeFile (node:fs:2119:14)
    at Object.queryNotionAndSaveResponse (/home/silvhua/repositories/notion/src/notionFunctions.js:60:14)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)
    at async main (evalmachine.<anonymous>:19:20) {
  code: 'ERR_INVALID_CALLBACK'
}


## restart kernel

In [1]:
console.log(`Running pipelineGetData.js`);
const {loadJsonFile, saveResponseJson} = require('../src/fileFunctions');
const nf = require('../src/notionFunctions');
// const period = 'month';
// const jsonFileName = `../data/notionTimeTracking_${period}`;
// const parsedJsonFileName = `../data/notion_time_tracking_parsed_${period}`;

// const response = await nf.queryNotionAndSaveResponse(period, jsonFileName, save=true);
// const data = loadJsonFile(`${jsonFileName}.json`);
// const parsedData = parseTimeTracking(data, save=true, parsedJsonFileName);
// console.log(`Done!`);


async function main() {
  const period = 'month';
  const jsonFileName = `../data/notionTimeTracking_${period}`;
  const parsedJsonFileName = `../data/notion_time_tracking_parsed_${period}`;

  const response = nf.queryNotionAndSaveResponse(period, jsonFileName, save=true);
  // const response = await nf.queryNotionAndSaveResponse(period, jsonFileName, save=true);
  // const data = loadJsonFile(`${jsonFileName}.json`);
  // const parsedData = parseTimeTracking(data, save=true, parsedJsonFileName);
  console.log(`Done!`);

}

main().catch(error => {
  console.error('Error:', error);
});

Running pipelineGetData.js
Current time stamp: 2023-12-26_1441
Done!


Promise { <pending> }

TypeError [ERR_INVALID_CALLBACK]: Callback must be a function. Received undefined
    at maybeCallback (node:fs:178:3)
    at Object.writeFile (node:fs:2119:14)
    at Object.queryNotionAndSaveResponse (/home/silvhua/repositories/notion/src/notionFunctions.js:60:14)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)

In [2]:
response

ReferenceError: response is not defined

## 1.2

In [1]:
console.log(`Running pipelineGetData.js`);
const {loadJsonFile, saveResponseJson} = require('../src/fileFunctions');
const nf = require('../src/notionFunctions');
// const period = 'month';
// const jsonFileName = `../data/notionTimeTracking_${period}`;
// const parsedJsonFileName = `../data/notion_time_tracking_parsed_${period}`;

// const response = await nf.queryNotionAndSaveResponse(period, jsonFileName, save=true);
// const data = loadJsonFile(`${jsonFileName}.json`);
// const parsedData = parseTimeTracking(data, save=true, parsedJsonFileName);
// console.log(`Done!`);


async function main() {
  const period = 'month';
  const jsonFileName = `../data/notionTimeTracking_${period}`;
  const parsedJsonFileName = `../data/notion_time_tracking_parsed_${period}`;

  const response = nf.queryNotionAndSaveResponse(period, jsonFileName, save=false);
  // const response = await nf.queryNotionAndSaveResponse(period, jsonFileName, save=true);
  // const data = loadJsonFile(`${jsonFileName}.json`);
  // const parsedData = parseTimeTracking(data, save=true, parsedJsonFileName);
  console.log(`Done!`);

}

main().catch(error => {
  console.error('Error:', error);
});

Running pipelineGetData.js
Current time stamp: 2023-12-26_1443
Done!


Promise { <pending> }

In [4]:
response

ReferenceError: response is not defined

# Update scripts to use pagination
https://developers.notion.com/reference/intro#pagination

## Iteration1

In [2]:
var {getCurrentTimestamp, loadJsonFile, saveResponseJson} = require('../src/fileFunctions');
var { Client } = require('@notionhq/client');
var fs = require('fs');
var {getTimestamp, addTimeDelta} = require('../src/notionFunctions');

async function queryNotionAndSaveResponse(
    period='week', jsonFileName='../data/notion_time_tracking', save = true, appendTimestamp = true) {
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
  
    if (period == 'past_week') {
      date_filter = {
        property: 'Created time',
        date: {past_week: {}},
      };
    } else if (period == 'quarter') {
      let start = getTimestamp('month', nMonths = 3);
      console.log(`Start date: ${start}. Period: ${period}`);
      date_filter = {
        and: [
          {property: 'Created time', date: {on_or_after: start}},
          {property: 'Created time', date: {before: addTimeDelta(start, period='month', nPeriod=3)}}
        ]
      };
    } else if (period == 'month' || period == 'week') {
        start = getTimestamp(period)
        date_filter = {
          and: [
            {property: 'Created time', date: {on_or_after: start}},
            {property: 'Created time', date: {before: addTimeDelta(start, period=period, nPeriod=1)}}
          ]
        };
    };
    const pages = [];
    let cursor = undefined;
    while (true) {
      const {results, next_cursor} = await notion.databases.query({
        database_id: process.env.notion_database,
        filter: date_filter,
        start_cursor: cursor
      });
      pages.push(...results);
      if (!next_cursor) {
        break
      };
      cursor = next_cursor
    }
    console.log(`${pages.length} issues successfully fetched.`)
  
  
    if (save) {
      let fileName = jsonFileName;
      // Add timestamp to the file name if appendTimestamp is true
      if (appendTimestamp) {
        const timestamp = getCurrentTimestamp();
        fileName = `${jsonFileName}_${timestamp}`;
      }
  
      // Save the response as a JSON file
      await fs.promises.writeFile(`${fileName}.json`, JSON.stringify(response, null, 2));
      
      console.log(`Response saved to ${jsonFileName}.json`);
    }
    return pages
  }

var period = 'month';
var jsonFileName = `../data/notionTimeTracking_${period}`;
var results = queryNotionAndSaveResponse(
  period, 
  jsonFileName,
  save=false
);

In [6]:
results

Promise {
  [
    {
      object: 'page',
      id: 'a7f579a6-5439-43b9-9e5d-f695889ac88f',
      created_time: '2023-12-01T04:26:00.000Z',
      last_edited_time: '2023-12-01T08:00:00.000Z',
      created_by: [Object],
      last_edited_by: [Object],
      cover: null,
      icon: null,
      parent: [Object],
      archived: false,
      properties: [Object],
      url: 'https://www.notion.so/2023-11-30-20-26-a7f579a6543943b99e5df695889ac88f',
      public_url: 'https://sand-gallimimus-e92.notion.site/2023-11-30-20-26-a7f579a6543943b99e5df695889ac88f'
    },
    {
      object: 'page',
      id: '3b00d2c2-7b81-4443-834b-84ed72b2ec7d',
      created_time: '2023-12-01T00:44:00.000Z',
      last_edited_time: '2023-12-01T04:26:00.000Z',
      created_by: [Object],
      last_edited_by: [Object],
      cover: null,
      icon: null,
      parent: [Object],
      archived: false,
      properties: [Object],
      url: 'https://www.notion.so/2023-11-30-16-44-ts-3b00d2c27b814443834b84ed72b2e

## 1.1

In [9]:
var {getCurrentTimestamp, loadJsonFile, saveResponseJson} = require('../src/fileFunctions');
var { Client } = require('@notionhq/client');
var fs = require('fs');
var {getTimestamp, addTimeDelta} = require('../src/notionFunctions');

async function queryNotionAndSaveResponse(
    period='week', jsonFileName='../data/notion_time_tracking', save = true, appendTimestamp = true) {
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
  
    if (period == 'past_week') {
      date_filter = {
        property: 'Created time',
        date: {past_week: {}},
      };
    } else if (period == 'quarter') {
      let start = getTimestamp('month', nMonths = 3);
      console.log(`Start date: ${start}. Period: ${period}`);
      date_filter = {
        and: [
          {property: 'Created time', date: {on_or_after: start}},
          {property: 'Created time', date: {before: addTimeDelta(start, period='month', nPeriod=3)}}
        ]
      };
    } else if (period == 'month' || period == 'week') {
        start = getTimestamp(period)
        date_filter = {
          and: [
            {property: 'Created time', date: {on_or_after: start}},
            {property: 'Created time', date: {before: addTimeDelta(start, period=period, nPeriod=1)}}
          ]
        };
    };
    const pages = [];
    let cursor = undefined;
    while (true) {
      const {results, next_cursor} = await notion.databases.query({
        database_id: process.env.notion_database,
        filter: date_filter,
        start_cursor: cursor
      });
      pages.push(...results);
      if (!next_cursor) {
        break
      };
      cursor = next_cursor
    }
    await console.log(`${pages.length} issues successfully fetched.`)
  
  
    if (save) {
      let fileName = jsonFileName;
      // Add timestamp to the file name if appendTimestamp is true
      if (appendTimestamp) {
        const timestamp = getCurrentTimestamp();
        fileName = `${jsonFileName}_${timestamp}`;
      }
  
      // Save the response as a JSON file
      await fs.promises.writeFile(`${fileName}.json`, JSON.stringify(pages, null, 2));
      
      console.log(`Response saved to ${jsonFileName}.json`);
    }
    return pages
  }

var period = 'month';
var jsonFileName = `../data/notionTimeTracking_${period}`;
var results = queryNotionAndSaveResponse(
  period, 
  jsonFileName,
  save=true
);

315 issues successfully fetched.
Response saved to ../data/notionTimeTracking_month.json


## 1.2 cleaned version

In [ ]:
var {getCurrentTimestamp, loadJsonFile, saveResponseJson} = require('../src/fileFunctions');
var { Client } = require('@notionhq/client');
var fs = require('fs');
var {getTimestamp, addTimeDelta} = require('../src/notionFunctions');

async function queryNotionAndSaveResponse(
    period='week', jsonFileName='../data/notion_time_tracking', save = true, appendTimestamp = true) {
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
  
    if (period == 'past_week') {
      date_filter = {
        property: 'Created time',
        date: {past_week: {}},
      };
    } else if (period == 'quarter') {
      let start = getTimestamp('month', nMonths = 3);
      console.log(`Start date: ${start}. Period: ${period}`);
      date_filter = {
        and: [
          {property: 'Created time', date: {on_or_after: start}},
          {property: 'Created time', date: {before: addTimeDelta(start, period='month', nPeriod=3)}}
        ]
      };
    } else if (period == 'month' || period == 'week') {
        start = getTimestamp(period)
        date_filter = {
          and: [
            {property: 'Created time', date: {on_or_after: start}},
            {property: 'Created time', date: {before: addTimeDelta(start, period=period, nPeriod=1)}}
          ]
        };
    };
    const pages = [];
    let cursor = undefined;
    while (true) {
      const {results, next_cursor} = await notion.databases.query({
        database_id: process.env.notion_database,
        filter: date_filter,
        start_cursor: cursor
      });
      pages.push(...results);
      if (!next_cursor) {
        break
      };
      cursor = next_cursor
    }
    await console.log(`${pages.length} issues successfully fetched.`)
  
  
    if (save) {
      let fileName = jsonFileName;
      // Add timestamp to the file name if appendTimestamp is true
      if (appendTimestamp) {
        const timestamp = getCurrentTimestamp();
        fileName = `${jsonFileName}_${timestamp}`;
      }
  
      // Save the response as a JSON file
      await fs.promises.writeFile(`${fileName}.json`, JSON.stringify(pages, null, 2));
      
      console.log(`Response saved to ${fileName}.json`);
    }
    return pages
  }

var period = 'month';
var jsonFileName = `../data/notionTimeTracking_${period}`;
var results = queryNotionAndSaveResponse(
  period, 
  jsonFileName,
  save=true
);

# *End of Page*